In [ ]:
%cd /content/ComfyUI

import os, json, requests, random, time
from urllib.parse import urlsplit

import torch
from PIL import Image
import numpy as np

from nodes import load_custom_node
from nodes import NODE_CLASS_MAPPINGS

load_custom_node("/content/ComfyUI/custom_nodes/ComfyUI-OmniGen")

LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
ailab_OmniGen = NODE_CLASS_MAPPINGS["ailab_OmniGen"]()

def download_file(url, save_dir, file_name):
    os.makedirs(save_dir, exist_ok=True)
    file_suffix = os.path.splitext(urlsplit(url).path)[1]
    file_name_with_suffix = file_name + file_suffix
    file_path = os.path.join(save_dir, file_name_with_suffix)
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as file:
        file.write(response.content)
    return file_path

@torch.inference_mode()
def generate(input):
    values = input["input"]

    image_1=values['image_1']
    if image_1:
        image_1=download_file(url=image_1, save_dir='/content/ComfyUI/input', file_name='image_1')
        image_1, _ = LoadImage.load_image(image_1)
    else:
        image_1 = None
    image_2=values['image_2']
    if image_2:
        image_2=download_file(url=image_2, save_dir='/content/ComfyUI/input', file_name='image_2')
        image_2, _ = LoadImage.load_image(image_2)
    else:
        image_2 = None
    image_3=values['image_3']
    if image_3:
        image_3=download_file(url=image_3, save_dir='/content/ComfyUI/input', file_name='image_3')
        image_3, _ = LoadImage.load_image(image_3)
    else:
        image_3 = None
    prompt = values['prompt']
    num_inference_steps = values['num_inference_steps']
    guidance_scale = values['guidance_scale']
    img_guidance_scale = values['img_guidance_scale']
    max_input_image_size = values['max_input_image_size']
    separate_cfg_infer = values['separate_cfg_infer']
    offload_model = values['offload_model']
    use_input_image_size_as_output = values['use_input_image_size_as_output']
    width = values['width']
    height = values['height']
    seed = values['seed']

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    image = ailab_OmniGen.generation(prompt, num_inference_steps, guidance_scale, img_guidance_scale, max_input_image_size, separate_cfg_infer, offload_model, use_input_image_size_as_output, width, height, seed, image_1=image_1, image_2=image_2, image_3=image_3)[0]
    return Image.fromarray(np.array(image*255, dtype=np.uint8)[0])

In [ ]:
input = { 
    "input": {
        "image_1": "https://litter.catbox.moe/j0onz6.jpg",
        "image_2": "https://litter.catbox.moe/t2xbql.webp",
        "image_3": "",
        "prompt": "A blond woman in wearing a long pink dress standing in a park. A woman is image_1 The pink dress is image_2.",
        "guidance_scale": 3.5,
        "img_guidance_scale": 1.8,
        "num_inference_steps": 50,
        "separate_cfg_infer": True,
        "offload_model": False,
        "use_input_image_size_as_output": False,
        "width": 1024,
        "height": 1024,
        "seed": 67551597694014,
        "max_input_image_size": 1024
    }
}
image = generate(input)
image